In [1]:
import pandas as pd
import requests as req
import numpy as np
import json
import time 
from lxml import etree
import os
import pickle as pkl
from pathlib import Path
from pandarallel import pandarallel
from tqdm import tqdm
tqdm().pandas()


0it [00:00, ?it/s]
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:656: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
output_path = Path.home() / "working_dir/cetesb/"
estacoes_filepath = "data/estacoes-cetesb.csv.gz"

In [3]:
#URI do WebService
URI_estacoes = "https://servicos.cetesb.sp.gov.br/arcgis/rest/services/QUALAR/CETESB_QUALAR/MapServer/6/query"

# IDs das estações
objectIds = np.arange(1, 100)
objectIds = ["{}".format(i) for i in objectIds]
objectIds = ", ".join(objectIds)

In [4]:
# Jogar o pedido para o site
params = {"f": "json", "returnGeometry": "true",
          "outFields": "*", "objectIds": objectIds}
r = req.get(URI_estacoes, params)
json_response = json.loads(r.text)

# Processar os dados e gerar um dataset
dataset_estacoes = []
for feature in json_response["features"]:
    estacao_info = {}
    estacao_info.update(feature["attributes"])
    estacao_info.update(feature["geometry"])
    dataset_estacoes.append(estacao_info)
dataset_estacoes = pd.DataFrame(dataset_estacoes)

# Salvar o resultado
dataset_estacoes.to_csv(estacoes_filepath, compression="gzip", index=False)

ConnectionError: HTTPSConnectionPool(host='servicos.cetesb.sp.gov.br', port=443): Max retries exceeded with url: /arcgis/rest/services/QUALAR/CETESB_QUALAR/MapServer/6/query?f=json&returnGeometry=true&outFields=%2A&objectIds=1%2C+2%2C+3%2C+4%2C+5%2C+6%2C+7%2C+8%2C+9%2C+10%2C+11%2C+12%2C+13%2C+14%2C+15%2C+16%2C+17%2C+18%2C+19%2C+20%2C+21%2C+22%2C+23%2C+24%2C+25%2C+26%2C+27%2C+28%2C+29%2C+30%2C+31%2C+32%2C+33%2C+34%2C+35%2C+36%2C+37%2C+38%2C+39%2C+40%2C+41%2C+42%2C+43%2C+44%2C+45%2C+46%2C+47%2C+48%2C+49%2C+50%2C+51%2C+52%2C+53%2C+54%2C+55%2C+56%2C+57%2C+58%2C+59%2C+60%2C+61%2C+62%2C+63%2C+64%2C+65%2C+66%2C+67%2C+68%2C+69%2C+70%2C+71%2C+72%2C+73%2C+74%2C+75%2C+76%2C+77%2C+78%2C+79%2C+80%2C+81%2C+82%2C+83%2C+84%2C+85%2C+86%2C+87%2C+88%2C+89%2C+90%2C+91%2C+92%2C+93%2C+94%2C+95%2C+96%2C+97%2C+98%2C+99 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f24e7d926d0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
# URL do site com as correspondências
URI_estacoes_2 = "http://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios.php"
r = req.get(URI_estacoes_2)
r.encoding = "utf-8"
raw_html = r.text
tree = etree.HTML(raw_html)
estacoes = tree.xpath("//select/option/@value")
estacoes_id2 = {}
for estacao in estacoes:
    (id2, name) = estacao.split("/")
    estacoes_id2[name] = id2
dataset_estacoes = pd.read_csv(estacoes_filepath, compression="gzip", na_values=["--"])
dataset_estacoes["ID2"] = None
for name, id2 in estacoes_id2.items():
    ind = (dataset_estacoes.Nome == name)
    dataset_estacoes.loc[ind, "ID2"] = id2    
    if (np.sum(ind) == 0):
        print("Não foi possível fazer a correspondência na estação {} ({})".format(name, id2))

ind = ~pd.isnull(dataset_estacoes.ID2)
dataset_estacoes = dataset_estacoes[ind]
dataset_estacoes.to_csv(estacoes_filepath, compression="gzip", index=False)

In [ ]:
# URI do site com dados horários
URI_dados_horarios = "https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_resultado.php"
URI_dados_horarios_meteo = "https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_Metrl.php"
# Carregar o arquivo com as informações sobre as estações
dataset_estacoes = pd.read_csv(estacoes_filepath, compression="gzip", na_values=["--"])

In [7]:
# Parâmetros livres
#inds = (dataset_estacoes.Municipio == "SAO PAULO")
#inds &= (dataset_estacoes.Nome == "Cerqueira César")
#inds &= (dataset_estacoes.Nome == "Pinheiros")
estacoes_id = dataset_estacoes.ID2.values

date_begin = "2019-01-01"
date_end = "2020-03-29"
date_range = pd.date_range(date_begin, date_end, freq="1d").sort_values(ascending=False)

In [8]:
def get_pollution_data(estacao_id, date):
    """
    Obtém a poluição na estação e data fornecidos
    """
    date_string = "{day:02d}/{month:02d}/{year:04d}".format(year=date.year, month=date.month, day=date.day)
    params = {"texData": date_string, "selEst": estacao_id}
    r = req.post(URI_dados_horarios, params)
    raw_html = r.text
    
    path = "{}_{}.html".format(estacao_id, date)
    
    with open(output_path / "raw_html" / path, 'w') as fid:
        fid.write(raw_html)

    return None

In [9]:
elements = pd.Series([(estacao_id, date) for date in date_range for estacao_id in estacoes_id])

In [ ]:
pollution_data = elements.progress_apply(lambda x: get_pollution_data(x[0], x[1]))

 60%|█████▉    | 3236/5429 [46:02<24:59,  1.46it/s]  

In [ ]:
pollution_data.to_pickle("aaa.pkl")

In [11]:
len(pollution_data)

NameError: name 'pollution_data' is not defined